In [1]:
import math
import numpy as np
import pandas as pd
import os

def obter_valor_float(mensagem):
    while True:
        try:
            return round(float(input(mensagem)), 3)
        except ValueError:
            print("Entrada inválida! Insira um número válido.")

def converter_medida(dp, unidade='mm'):
    if unidade == 'mm':
        resposta = input("Deseja converter mm para polegadas? (sim/não): ").strip().lower()
        if resposta == 'sim':
            dp = round(dp / 25.4, 3)
            print(f"Diâmetro convertido para polegadas: {dp:.3f} in")
        else:
            print(f"Diâmetro mantido em milímetros: {dp:.3f} mm")
    elif unidade == 'in':
        resposta = input("Deseja converter polegadas para milímetros? (sim/não): ").strip().lower()
        if resposta == 'sim':
            dp = round(dp * 25.4, 3)
            print(f"Diâmetro convertido para milímetros: {dp:.3f} mm")
        else:
            print(f"Diâmetro mantido em polegadas: {dp:.3f} in")
    return dp

def calcular_afastamento(dp, PE, pr):
    return round(0.0123 * (2 * (PE / pr) + 1.5) * dp, 3)

def calcular_espacamento(hb, a):
    y = round(hb / a, 3)
    tipo_bancada = "BANCADA BAIXA" if y <= 4 else "BANCADA ALTA"
    print(f"Razão Y: {y:.3f}, Tipo: {tipo_bancada}")
    resposta = input("Os furos são instantâneos ou com retardos? ").strip().lower()
    if resposta == 'instantâneos':
        e = round(0.33 * (hb + 2 * a) if tipo_bancada == "BANCADA BAIXA" else 2 * a, 3)
    elif resposta == 'retardos':
        e = round((hb + 7 * a) / 8 if tipo_bancada == "BANCADA BAIXA" else 1.4 * a, 3)
    else:
        print("Opção inválida! Usando valor padrão.")
        e = 0
    return e

def calcular_parametros(hb, a, alpha, dp, PE, pr, e):
    sf = round(0.3 * a, 3)
    hf = round((hb / math.cos(math.radians(alpha))) + ((1 - (alpha / 100)) * sf), 3)
    t = round(0.7 * a, 3)
    vf = round(hb * a * e, 3)
    pe_calc = round(hf / vf, 3)
    return sf, hf, t, vf, pe_calc

def calcular_rlc(dp, PE):
    rlc = (np.pi * dp**2 / 4000) * PE
    print(f"RLC calculado: {rlc:.3f} kg/m")
    return round(rlc, 3)

def main():
    print("Bem-vindo ao projeto do plano de fogo!")

    hb = obter_valor_float("Insira a altura da bancada: ")
    dp = obter_valor_float("Insira o diâmetro da perfuração: ")
    dp = converter_medida(dp)
    alpha = obter_valor_float("Insira o ângulo da inclinação dos furos: ")
    PE = obter_valor_float("Insira a densidade do explosivo (ton/m³): ")
    pr = obter_valor_float("Insira a densidade da rocha (ton/m³): ")

    a = calcular_afastamento(dp, PE, pr)
    e = calcular_espacamento(hb, a)
    sf, hf, t, vf, pe_calc = calcular_parametros(hb, a, alpha, dp, PE, pr, e)
    hct = round(hf - t, 3)
    rlc = calcular_rlc(dp, PE)
    ce = round(rlc * hct, 3)
    rc = round(ce / vf, 3)

    ap = obter_valor_float("Insira o valor dos Acessórios da perfuratriz (R$/m): ")
    mo = obter_valor_float("Insira o valor da mão de obra (R$/m): ")
    cec = obter_valor_float("Insira o valor do custo do equipamento e compressor (R$/m): ")
    cgl = obter_valor_float("Insira o valor do combustível, graxas e lubrificantes (R$/m): ")
    anfo = obter_valor_float("Insira o valor do ANFO (R$/kg): ")
    espoletas = obter_valor_float("Insira o valor da espoleta (R$/unidade): ")
    ret = obter_valor_float("Insira o valor dos retardos de superfície de 30 ms (R$/unidade): ")
    cd = obter_valor_float("Insira o valor do Cordel detonante (R$/m): ")
    ee = obter_valor_float("Insira o valor do Estopim espoletado (R$/unidade): ")

    volume_a_desmontar = obter_valor_float("Insira o valor do desmonte (m³): ")
    retardos = obter_valor_float("Insira a quantidade de retardos: ")
    metros_de_cordel = obter_valor_float("Insira a quantidade de Metros de Cordel: ")
    quantidade_de_estopim = obter_valor_float("Insira a quantidade de estopim: ")

    nf = round(volume_a_desmontar / vf)
    mp = round(hf * nf, 3)
    te = round(ce * nf, 3)
    ctp = round(mp * (ap + mo + cec + cgl), 3)
    cte = round(anfo * te, 3)
    cta = round((espoletas * nf) + (ret * retardos) + (cd * metros_de_cordel) + (ee * quantidade_de_estopim), 3)
    ctd = round(ctp + cte + cta, 3)
    c_m3 = round(ctd / volume_a_desmontar, 3)
    custo_por_tonelada = round(c_m3 / pr, 3)

    print(f"Número de furos: {nf}")
    print(f"Total de metros perfurados: {mp:.3f} metros")
    print(f"Total de explosivos utilizados: {te:.3f} kg")
    print(f"Custo total de perfuração: {ctp:.3f} reais")
    print(f"Custo total de explosivos: {cte:.3f} reais")
    print(f"Custo total de acessórios: {cta:.3f} reais")
    print(f"Custo total do desmonte: {ctd:.3f} reais")
    print(f"Custo total do metro cúbico: {c_m3:.3f} reais")
    print(f"Custo total por tonelada: {custo_por_tonelada:.3f} reais")

    resultados = {
        "Afastamento (a)": a,
        "Espaçamento (e)": e,
        "Subfuração (sf)": sf,
        "Profundidade do furo (hf)": hf,
        "Tampão (t)": t,
        "Volume desmontado (vf)": vf,
        "Perfuração específica (PE calc)": pe_calc,
        "Altura da carga (HCT)": hct,
        "RLC": rlc,
        "Carga de explosivo por furo (CE)": ce,
        "Razão de carregamento (RC)": rc,
        "Número de furos (NF)": nf,
        "Metros perfurados (MP)": mp,
        "Explosivo total (TE)": te,
        "Custo perfuração (CTP)": ctp,
        "Custo explosivo (CTE)": cte,
        "Custo acessórios (CTA)": cta,
        "Custo total desmonte (CTD)": ctd,
        "Custo por m³": c_m3,
        "Custo por tonelada": custo_por_tonelada
    }

    df = pd.DataFrame([resultados])
    nome_arquivo = "relatorio_plano_de_fogo.xlsx"
    df.to_excel(nome_arquivo, index=False)
    print(f"Relatório salvo em {nome_arquivo}")
    os.startfile(nome_arquivo)

if __name__ == "__main__":
    main()

    

Bem-vindo ao projeto do plano de fogo!
Insira a altura da bancada: 15
Insira o diâmetro da perfuração: 101
Deseja converter mm para polegadas? (sim/não): não
Diâmetro mantido em milímetros: 101.000 mm
Insira o ângulo da inclinação dos furos: 20
Insira a densidade do explosivo (ton/m³): 0.85
Insira a densidade da rocha (ton/m³): 2.70
Razão Y: 5.669, Tipo: BANCADA ALTA
Os furos são instantâneos ou com retardos? retardos
RLC calculado: 6.810 kg/m
Insira o valor dos Acessórios da perfuratriz (R$/m): 0.81
Insira o valor da mão de obra (R$/m): 1.50
Insira o valor do custo do equipamento e compressor (R$/m): 2.01
Insira o valor do combustível, graxas e lubrificantes (R$/m): 1.20
Insira o valor do ANFO (R$/kg): 0.90
Insira o valor da espoleta (R$/unidade): 6
Insira o valor dos retardos de superfície de 30 ms (R$/unidade): 6
Insira o valor do Cordel detonante (R$/m): 0.45
Insira o valor do Estopim espoletado (R$/unidade): 0.80
Insira o valor do desmonte (m³): 4481
Insira a quantidade de retardo